In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 21.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import os

In [5]:
df = pd.read_csv('/content/drive/MyDrive/csv/combine.csv')

In [20]:
# title 및 review_text 열만 선택하여 새로운 데이터프레임 생성
review = df[['title', 'review_text']].copy()

# 결과 확인
print(review)

          title                                        review_text
0      로니로티 건대점  {1: '양이 작을까봐 큰 사이즈로 시켰는데 정말 대박 많이 주네요. 둘이서 다 먹...
1           도우터  {1: '건대 브런치 도우터 분위기도좋고 플레이트 비주얼짱\n너무예쁘고 맛있어요 \...
2    오코노미야키식당하나  {1: '일본인 친구가 소개시켜준 한국맛집🧡\n그리고 사장님과 직원분들도 일본인 이...
3      화기애애 건대점  {1: '고기 넘 맛나서 재방문했어요.\n셀프바에 버섯도 있어서 너무 맘에들더라구요...
4        시옌 건대점  {1: '건대에 양꼬치나 훠궈 파는 곳은 많은데 이런 짜장 짬뽕 탕수육 파는 곳이 ...
..          ...                                                ...
266  김밥천국 구의2호점  {1: '고추김밥을 좋아하는 사람으로써 고추 좀 많이 넣어주세요 라고 했더니 고추 ...
267         소바쿠  {1: '맛있어요~\n\n매번 혼자 오다가, 이번에는 여러 명이 와서 이것 저것 주...
268       찐족발보쌈  {1: '저는 족발, 보쌈세트 4만원짜리\n포장해서 먹었어요~^^\n\n그 전에 런...
269      홍성토종한우  {1: '친절하고 고기도 좋아요', 2: '좋아요', 3: '소고기랑 닭갈비🥰\n닭...
270         산풀잎  {1: '조금 비싸지만 \n수제이기에 \n건강에도 좋고 \n맛도 좋고', 2: '커...

[271 rows x 2 columns]


## 맛에 관한 내용

In [ ]:
from konlpy.tag import Okt
import pandas as pd

def extract_taste_related_keywords(text):
    if pd.isna(text) or text == '':
        return []

    okt = Okt()
    taste_keywords = []
    taste_adjectives = [word for word, pos in okt.pos(text) if pos == 'Adjective' and ('맛' in word or '맛있' in word or '맛없' in word)]
    taste_keywords.extend(taste_adjectives)
    return taste_keywords

# 'flavor' 열 초기화
review['flavor'] = ''

# 각 행의 리뷰에서 맛에 관련된 키워드 추출하여 'flavor' 열에 추가
for index, row in review.iterrows():
    text = row['review_text']
    if isinstance(text, dict):
        text = ' '.join(text.values())
    keywords = extract_taste_related_keywords(text)
    review.at[index, 'flavor'] = ', '.join(keywords)

# 결과 확인
print(review)

KeyboardInterrupt: 

In [ ]:
# 긍정적인 키워드 리스트
positive_words = ['있']

# 부정적인 키워드 리스트
negative_words = ['없']

# 'flavor_all', 'flavor_positive', 'flavor_negative' 열 초기화
review['flavor_all'] = 0
review['flavor_positive'] = 0
review['flavor_negative'] = 0

# 각 행에서 맛에 관련된 키워드를 기반으로 전체, 긍정적인, 부정적인 키워드 개수 세기
for index, row in review.iterrows():
    flavor_text = row['flavor']
    flavor_all = 0
    flavor_positive = 0
    flavor_negative = 0
    if isinstance(flavor_text, str):
        flavor_all = len(flavor_text.split(', '))
        for keyword in flavor_text.split(', '):
            if any(word in keyword for word in positive_words):
                flavor_positive += 1
            elif any(word in keyword for word in negative_words):
                flavor_negative += 1
    review.at[index, 'flavor_all'] = flavor_all
    review.at[index, 'flavor_positive'] = flavor_positive
    review.at[index, 'flavor_negative'] = flavor_negative

# 결과 확인
print(review)

In [ ]:
review.to_csv('/content/drive/MyDrive/csv/flavor.csv', index=False)

## 분위기에 관한 내용

In [ ]:
from konlpy.tag import Okt
import pandas as pd

def extract_atmosphere_related_keywords(text):
    if pd.isna(text) or text == '':
        return []

    okt = Okt()
    keywords = []

    # 명사와 형용사 추출
    nouns_and_adjectives = okt.nouns(text) + [word for word, pos in okt.pos(text) if pos == 'Adjective']

    # 분위기와 관련된 키워드 필터링
    atmosphere_related_keywords = [word for word in set(nouns_and_adjectives) if word in ['인테리어', '매장', '크기', '혼밥', '단체모임', '뷰', '아늑하다', '야외', '공간',
                                                  '멋지다', '컨셉', '독특하다', '분위기', '사진', '대화', '음악', '집중', '혼술',
                                                  '로맨틱', '아늑', '모던', '트렌디', '편안', '고급스러운', '캐주얼']]

    return ', '.join(atmosphere_related_keywords)

# 'review_text' 열 각 행에 대해 분위기와 관련된 키워드 추출하여 새로운 열에 저장
review['mood_keyword'] = ''
for index, row in review.iterrows():
    text = row['review_text']
    if isinstance(text, dict):
        text = ' '.join(text.values())
    keywords = extract_atmosphere_related_keywords(text)
    review.at[index, 'mood_keyword'] = ', '.join(keywords)

# 결과 확인
print(review['mood_keyword'])

0                        매, 장, ,,  , 사, 진, ,,  , 분, 위, 기
1      매, 장, ,,  , 인, 테, 리, 어, ,,  , 공, 간, ,,  , 음, 악...
2                        매, 장, ,,  , 사, 진, ,,  , 분, 위, 기
3            매, 장, ,,  , 사, 진, ,,  , 분, 위, 기, ,,  , 대, 화
4                                                   사, 진
                             ...                        
266                                     혼, 밥, ,,  , 사, 진
267          컨, 셉, ,,  , 혼, 밥, ,,  , 매, 장, ,,  , 분, 위, 기
268                         혼, 밥, ,,  , 매, 장, ,,  , 사, 진
269                                                 크, 기
270                         뷰, ,,  , 분, 위, 기, ,,  , 대, 화
Name: mood_keyword, Length: 271, dtype: object


## 가성비에 관한 내용

In [21]:
import re

# 가성비 관련 문구를 추출하는 함수
def extract_flex_phrases(text):
    matches = re.findall(r'가성비\s*(?:[가-힣a-zA-Z\s]+)\s*(?:[너무넘]|\s+)?', text)
    return matches

# 새로운 열을 만들어서 각 행의 가성비 관련 문구를 저장할 리스트 초기화
review['flex'] = ''

# 각 review_text 셀을 순회하며 가성비 관련 문구 추출하여 리스트에 저장
for index, row in review.iterrows():
    text = row['review_text']
    if isinstance(text, dict):
        # 딕셔너리 형태의 텍스트를 문자열로 변환
        text = ' '.join(text.values())
    # 가성비 관련 문구 추출
    flex_phrases = extract_flex_phrases(text)
    # 추출된 문구를 flex 열에 추가
    review.at[index, 'flex'] = flex_phrases

# 결과 확인
print(review['flex'])


0      [가성비 끝판왕, 가성비좋고 음식도 빨리 나오고 친절하고 항상 가는곳, 가성비 , ...
1                                                     []
2                                                     []
3      [가성비 최고 네요, 가성비 최고 고기집, 가성비 갑 고깃집이에용, 가성비좋고 분위...
4                                                     []
                             ...                        
266                                     [가성비가 좋고 맛이 좋아요]
267                                                   []
268    [가성비 최고입니다 , 가성비 괜찮아요, 가성비 좋아요, 가성비 굿굿, 가성비짱, ...
269                                                   []
270                                                   []
Name: flex, Length: 271, dtype: object


In [30]:
# 리스트에서 부정적인 표현을 포함한 요소의 수를 세는 함수
def count_negative_phrases(phrases, negative_phrases):
    negative_count = 0
    for phrase in phrases:
        for negative_phrase in negative_phrases:
            if negative_phrase in phrase:
                negative_count += 1
                break  # 하나의 요소에 대해 한 번 발견되면 더 이상 검사하지 않음
    return negative_count

# 부정적인 표현 리스트
negative_phrases = ['별로', '안', '실망', '불만족', '아쉽', '글쎄', '없', '떨',
                    '꽝', '않', '아쉬', '그닥', '아니', '최악', '못']

# 새로운 열을 만들어서 각 행의 가성비 관련 문구를 저장할 리스트 초기화
review['flex_count'] = 0
review['flex_negative_count'] = 0

# 각 행에 대해 처리
for index, row in review.iterrows():
    phrases = row['flex']
    if isinstance(phrases, list):
        # 전체 요소 개수를 세어서 flex_count 열에 추가
        review.at[index, 'flex_count'] = len(phrases)
        # 부정적인 표현을 포함한 요소의 수를 세어서 flex_negative_count 열에 추가
        review.at[index, 'flex_negative_count'] = count_negative_phrases(phrases, negative_phrases)

# flex_positive_count 계산하여 추가
review['flex_positive_count'] = review['flex_count'] - review['flex_negative_count']

# 결과 확인
print(review)


          title                                        review_text  \
0      로니로티 건대점  {1: '양이 작을까봐 큰 사이즈로 시켰는데 정말 대박 많이 주네요. 둘이서 다 먹...   
1           도우터  {1: '건대 브런치 도우터 분위기도좋고 플레이트 비주얼짱\n너무예쁘고 맛있어요 \...   
2    오코노미야키식당하나  {1: '일본인 친구가 소개시켜준 한국맛집🧡\n그리고 사장님과 직원분들도 일본인 이...   
3      화기애애 건대점  {1: '고기 넘 맛나서 재방문했어요.\n셀프바에 버섯도 있어서 너무 맘에들더라구요...   
4        시옌 건대점  {1: '건대에 양꼬치나 훠궈 파는 곳은 많은데 이런 짜장 짬뽕 탕수육 파는 곳이 ...   
..          ...                                                ...   
266  김밥천국 구의2호점  {1: '고추김밥을 좋아하는 사람으로써 고추 좀 많이 넣어주세요 라고 했더니 고추 ...   
267         소바쿠  {1: '맛있어요~\n\n매번 혼자 오다가, 이번에는 여러 명이 와서 이것 저것 주...   
268       찐족발보쌈  {1: '저는 족발, 보쌈세트 4만원짜리\n포장해서 먹었어요~^^\n\n그 전에 런...   
269      홍성토종한우  {1: '친절하고 고기도 좋아요', 2: '좋아요', 3: '소고기랑 닭갈비🥰\n닭...   
270         산풀잎  {1: '조금 비싸지만 \n수제이기에 \n건강에도 좋고 \n맛도 좋고', 2: '커...   

                                                  flex  flex_count  \
0    [가성비 끝판왕, 가성비좋고 음식도 빨리 나오고 친절하고 항상 가는곳, 가성비 , ...          19   
1                  

In [31]:
review.to_csv('/content/drive/MyDrive/csv/flex.csv', index=False)